In [27]:
from utils import *
from variables import *
from datetime import datetime, timedelta
import requests
import re
from scipy.stats import norm
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pyhtml2pdf import  converter
import shutil
from plotly.colors import n_colors
from typing import Union, List, Dict
from bs4 import BeautifulSoup
import traceback


In [37]:
try:
    1/0
except:
    print(traceback.format_exc())

Traceback (most recent call last):
  File "C:\Users\Vinicius\AppData\Local\Temp\ipykernel_948\3942357433.py", line 2, in <module>
    1/0
ZeroDivisionError: division by zero



In [35]:
e

NameError: name 'e' is not defined

{0: 'DAX', 1: 'STOXX'}

In [13]:
auswahl = 1


info@dieball.de


In [3]:
def get_date_idx(list_idxs)->list:
    while(True):
        date_idxs = [0, 1]
        date_idxs = list(map(int,date_idxs))
        date_idxs = [idx for idx in date_idxs if idx in list_idxs]
        if len(date_idxs) < 1:
            print("No valid indexes passed.")
        else:
            return date_idxs
        

def get_heute():
    heute = datetime.today()
    if heute.weekday() > 4:
        #sunday => wd = 6 -> td = +1
        #sartuday => wd = 5 > td +2
        heute += timedelta(days= 7 - heute.weekday())
    print(f'heute = {heute.strftime("%d/%m/%Y")}')
    return heute



def get_euribor_3m()->float:
    euribor_3m_df = pd.read_html("https://www.euribor-rates.eu/en/current-euribor-rates/2/euribor-rate-3-months/")[0].rename(columns= {0: "Date", 1 : "InterestRate"})
    euribor_row = euribor_3m_df.loc[0]
    print(f"Euribor 3M in {euribor_row['Date']} is {euribor_row['InterestRate']}")
    return float(euribor_row["InterestRate"].replace("%",""))/100

def get_finazen_price(stock_idx)->float:
    dict_stock_names = {
    0 : 'vdax-new-2m',
    1 : 'vstoxx'
    }
    stock = dict_stock_names[stock_idx]
    #accepted values for stock: , 
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    }

    response = requests.get('https://www.finanzen.net/index/' + stock, headers=headers)
    html = BeautifulSoup(response.content, 'html.parser')
    aux = BeautifulSoup.findAll(html,id = 'snapshot-value-fst-current-0')[0].span.get_text()
    price = re.findall(r'\d{0,},\d{0,}',aux)[0]
    price = float(price.replace(',','.'))
    print(f"{stock} = {price} PTS")
    return price



def get_overview(auswahl):
    ## Download tradingDates and future_date_col
    global url, headers, tradingDates, future_date_col, overview_df
    if auswahl == 1:
    # STOXX
        url = 'https://www.eurex.com/api/v1/overallstatistics/69660'
        print("STOXX")
    else:
    # DAX
        url = 'https://www.eurex.com/api/v1/overallstatistics/70044'
        print ("DAX")
        
    headers = {'Accept': '*/*'}
    params_ov = {'filtertype': 'overview', 'contracttype': 'M'}
    print('Getting data from EUREX...')
    response = requests.get(url, params=params_ov, headers=headers,timeout = 8)

    if not response.ok:
        raise ValueError("Conection failed.")

    tradingDates = pd.to_datetime(pd.Series(response.json()['header']['tradingDates']),format="%d-%m-%Y %H:%M")

    print("Index     Date")
    for idx,date in enumerate(tradingDates.dt.strftime("%d-%m-%Y")):
        print(f" {idx:<6} {date:^3}")

    #date_idxs = get_date_idx(tradingDates.index)
    params_ov['busdate'] = tradingDates[0].strftime("%Y%m%d")
    response = requests.get(url, params=params_ov, headers=headers,timeout = 8)
    if not response.ok:
            raise ValueError("Conection failed.")

    params_details = {}
    if 'dataRows' in response.json(): 
        overview_df = pd.DataFrame(response.json()['dataRows'])
        overview_df = overview_df[overview_df.contractType == 'M']
        overview_df.sort_values('date',ignore_index=True,inplace=True)
        future_date_col = overview_df.loc[:,'date']
        overview_df.date = pd.to_datetime(overview_df.date,format='%Y%m%d').dt.strftime('%d-%m-%Y')

    else:
        raise ValueError

    params_details['busdate'] = f"{tradingDates[1].strftime('%Y%m%d')}"
    response = requests.get(url, params=params_details, headers=headers,timeout = 8)
    if not response.ok:
        raise ValueError("Conection failed.")
    
def get_contracts():
    params_details = {}
    params_details['filtertype'] = 'detail'
    params_details['contracttype'] = 'M'
    global dict_prod_bus
    dict_prod_bus = {}
    for productdate_idx in [0,1]:
        dict_prod_bus[productdate_idx] = {}
        params_details['productdate'] = future_date_col[productdate_idx]
        for busdate_idx in [0,1]:
            params_details['busdate'] = f"{tradingDates[busdate_idx].strftime('%Y%m%d')}" 
            print(f"busdate_idx = {busdate_idx}   | productdate_idx = {productdate_idx}")
            
            if (busdate_idx == 1) and (productdate_idx == 1):
                # No need to request data for this case so we skip this iteration
                continue
            
            # Request data
            response = requests.get(url, params=params_details, headers=headers, timeout = 10)
            if not response.ok:
                raise ValueError("Conection failed.")
            contractsCall_aux_df =  pd.DataFrame(response.json()['dataRowsCall'])
            contractsPut_aux_df =  pd.DataFrame(response.json()['dataRowsPut'])

            # Create dataframe with the just requested data

            aux_df = contractsCall_aux_df[['strike','openInterest']].merge(
                contractsPut_aux_df[['strike','openInterest']],
                on = "strike",
                how="left",
                suffixes=('_CF', '_PF')
            )
            dict_prod_bus[productdate_idx][busdate_idx] = aux_df

In [4]:
list_emails

[{'from': 'almeidabarrosdf@gmail.com',
  'mailto': ['MyTest20230902@outlook.de', 'dummy_eurex@outlook.com'],
  'subject': 'OI-Tabelle',
  'text': 'dummy_msg.txt',
  'condition': 'always'},
 {'from': 'detlefd@gmx.net',
  'mailto': ['MyTest20230902@outlook.de',
   'dummy_eurex@outlook.com',
   'almeidabarrosdf@gmail.com'],
  'subject': 'OI-Tabelle',
  'text': 'dummy_msg.txt',
  'condition': 'always'},
 {'from': 'info@dieball.de',
  'mailto': ['almeidabarrosdf@gmail.com'],
  'subject': 'OI-Tabelle',
  'text': 'dummy_msg.txt',
  'condition': 'always'}]

In [4]:
auswahl = 1
heute = None

In [5]:
get_contracts()


STOXX
Getting data from EUREX...
Index     Date
 0      01-09-2023
 1      31-08-2023
 2      30-08-2023
 3      29-08-2023
 4      28-08-2023
 5      25-08-2023
 6      24-08-2023
 7      23-08-2023
 8      22-08-2023
 9      21-08-2023
 10     18-08-2023
 11     17-08-2023
 12     16-08-2023
 13     15-08-2023
 14     14-08-2023
 15     11-08-2023
 16     10-08-2023
 17     09-08-2023
 18     08-08-2023
 19     07-08-2023
 20     04-08-2023
busdate_idx = 0   | productdate_idx = 0
busdate_idx = 1   | productdate_idx = 0
busdate_idx = 0   | productdate_idx = 1
busdate_idx = 1   | productdate_idx = 1


In [6]:
"""
auswahl: Option defined by the user. See variables.auswahl_dict.
heute: it should be a date as string using dd/mm/yyyy format. If None, it will be defined by get_heute().
"""
get_overview(auswahl)
# First stage
# Create results folder in case they still were not created
list_folder_results =  [current_results_path, old_results_path]
for folder in list_folder_results:
    create_folder(folder)

# During the generation of the PDF, the CSS file must be in the same directory
# as the generated html file. So in case the CSS file is still not the in the
# Results folder, we copy this file to it
if not os.path.exists(result_css):
    shutil.copyfile(src_css, result_css)

#auswahl = int(input("DAX = 0; STOXX = 1   :"))


# **tbd: heute shall be a working day (Mo - Fr)**
expiry = datetime.strptime(future_date_col[0],"%Y%m%d")
expiry_1 = datetime.strptime(future_date_col[1],"%Y%m%d")

if not heute:
    heute = get_heute()
else:
    heute = datetime.strptime(heute,"%d/%m/%Y")
tage_bis_verfall = (expiry - heute).days +1

heute = 04/09/2023


In [19]:
# WAS CHANGED
stocks = pd.read_html("https://www.boerse-stuttgart.de/en/")

auswahl = 1
if auswahl == 0:
    stocks_df = stocks[0]
    stock_price = stocks_df.loc[stocks_df['Indices GER'] == "L&S DAX","Price"].values[0]
    Spannweite = 2000
    Schritt = 50
    VDAX_Laufzeit = 60


else:
    stocks_df = stocks[1]
    stock_price = stocks_df.loc[stocks_df['Indices EU / USA / INT'] == "CITI Euro Stoxx 50","Price"].values[0]
    Spannweite = 700
    Schritt = 25
    # ASK: VDAX for both?
    VDAX_Laufzeit = 365


span = (Spannweite/4)
ZentralKurs = round(stock_price/span)*span



Minkurs = ZentralKurs - (Spannweite/ 2)
Maxkurs = Minkurs + Spannweite
Schritte = int(Spannweite / Schritt)

DetailMin = round(stock_price - (Spannweite)/4)
DetailMax = DetailMin + (Spannweite/2)


if tage_bis_verfall >= 1:
    delta = 0.5
else:
    delta = 1

print(f"stock_price = {stock_price}")
print(f"Minkurs = {Minkurs}")
print(f"Maxkurs = {Maxkurs}")
print(f"Schritte = {Schritte}")
print(f"ZentralKurs = {ZentralKurs}")
print(f"VDAX_Laufzeit = {VDAX_Laufzeit}")

stock_price = 4291.2
Minkurs = 4025.0
Maxkurs = 4725.0
Schritte = 28
ZentralKurs = 4375.0
VDAX_Laufzeit = 365


In [8]:
# Second stage
# Create Basis column that will be used for multiple tables
Basis = pd.Series((Minkurs + np.arange(int(Schritte) +1)* Schritt)[::-1])

# Transform series into dataframe to make dataframe creation easier 
Basis_df = Basis.reset_index().rename(columns={0:"Basis"})[["Basis"]]


# Ueberhaenge_df, Summery_df are created from a copy of the same Dataframe
# so they have the same index. It means they could be in one  Dataframe but
#  I decided to keep them separated as they were in the VBA code.

Ueberhaenge_df = Basis_df.copy()
Summery_df = Basis_df.copy()


for productdate_idx in [0,1]:
    for busdate_idx in [0,1]:
        
        if (busdate_idx == 1) and (productdate_idx == 1):
            continue

        # Create dataframe with the just requested data
        aux_df = Basis_df.copy()

        aux_df = aux_df.merge(
            dict_prod_bus[productdate_idx][busdate_idx],
            left_on = "Basis",
            right_on = "strike",
            how= "left"
        )

        if productdate_idx == 0:
            Ueberhaenge_df[busdate_idx] = aux_df.openInterest_PF - aux_df.openInterest_CF
            if busdate_idx == 0:
                Summery_df["openInterest_PF"] = aux_df["openInterest_PF"]
                Summery_df["openInterest_CF"] = aux_df["openInterest_CF"]
        
        elif productdate_idx == 1:
            if busdate_idx == 0:
                Ueberhaenge_df["nextContract"] = aux_df.openInterest_PF - aux_df.openInterest_CF
                Summery_df["nextContract"] = Ueberhaenge_df["nextContract"]


Ueberhaenge_df.rename(columns = {0 : "Front"},inplace=True)
Summery_df["heute"] = Ueberhaenge_df["Front"] * 0.2* delta

Ueberhaenge_df.rename(columns = {1 : "Last"},inplace=True)
Summery_df["last_day"] = Ueberhaenge_df["Last"] * 0.2* delta


Ueberhaenge_df["Summe"] = Ueberhaenge_df[["Front", "nextContract"]].sum(axis=1)
Ueberhaenge_df = Ueberhaenge_df[['Basis', 'Summe',"Last",'Front', "nextContract"]]


Summery_df["Änderung"] = Summery_df.heute - Summery_df.last_day

if (delta == 1):
    Summery_df['nextContract'] =  Summery_df['nextContract'] / 2

SummeryDetail_df = Summery_df[(Summery_df.Basis >= DetailMin) & (Summery_df.Basis < (DetailMax + Schritt))]

In [22]:
# Third stage
SchrittWeite = 10
InterestRate = get_euribor_3m()

# ASK: Is VDAX used for both? Should it be divided by 100
VDAX = get_finazen_price(auswahl)/100

Tage = tage_bis_verfall
if Tage == 0:
    Tage = 0.5

Tage_1 = (expiry_1 - heute).days +1
Kurs_count = int(Spannweite/SchrittWeite)+1 
HedgeBedarf_kurs=  pd.DataFrame(Maxkurs - np.arange(Kurs_count)* SchrittWeite,columns= ["Basis"])


Hedge_dimensions = Kurs_count,int(Schritte+1)
HedgeBedarf_values = np.zeros(Hedge_dimensions)
HedgeBedarf1_values = np.zeros(Hedge_dimensions)


for k in range(Hedge_dimensions[1]):
    Basis_value = Basis[k]
    Kontrakte = Ueberhaenge_df.loc[k,"Front"]
    Kontrakte_1 = Ueberhaenge_df.loc[k,"nextContract"]
    for i in range(Hedge_dimensions[0]):
        Kurs = Maxkurs - i * SchrittWeite
        # In python np.log = natural log
        h1 = np.log(Kurs / Basis_value)
        sigma = VDAX * ((Tage / VDAX_Laufzeit) ** 0.5)
        sigma_1 = VDAX * ((Tage_1 / VDAX_Laufzeit) ** 0.5)
        h2 = InterestRate + sigma * sigma / 2
        h2_1 = InterestRate + sigma_1 * sigma_1 / 2
        d1 = (h1 + (h2 * (Tage / 365))) / (sigma * ((Tage / 365) ** 0.5))

        d1_1 = (h1 + (h2_1 * (Tage_1 / 365))) / (sigma_1 * ((Tage_1 / 365) ** 0.5))

        Phi = norm.pdf(d1, 0, 1)               
        Phi_1 = norm.pdf(d1_1, 0, 1)

        Gamma = Phi / (Kurs * (sigma * (Tage / 365) ** 0.5))
        Gamma_1 = Phi_1 / (Kurs * (sigma_1 * (Tage_1 / 365) ** 0.5))
        HedgeBedarf_values[i,k] = Gamma * Kontrakte / 5
        HedgeBedarf1_values[i,k] = Gamma_1 * Kontrakte_1 / 5


HedgeSum = HedgeBedarf_values.sum(axis=1)/2
HedgeSum_1 = HedgeBedarf1_values.sum(axis=1)/2
HedgeBedarf_df = pd.DataFrame(data =HedgeBedarf_values, columns=Basis)
HedgeSum_df = pd.DataFrame(HedgeSum,columns=["HedgeSum"])
HedgeBedarf_df = pd.concat([HedgeBedarf_kurs,HedgeSum_df,HedgeBedarf_df], axis=1)

HedgeBedarf1_df = pd.DataFrame(data =HedgeBedarf1_values, columns=Basis)
HedgeSum1_df = pd.DataFrame(HedgeSum_1,columns=["HedgeSum"])
HedgeBedarf1_df = pd.concat([HedgeBedarf_kurs,HedgeSum1_df,HedgeBedarf1_df], axis=1)

# Fourth Stage


info_df = pd.DataFrame({
    'ZentralKurs' : [ZentralKurs],
    'Spannweite': [Spannweite],
    'SchrittWeite': [SchrittWeite],
    'Schritt' : [Schritt],
    'auswahl': auswahl_dict[auswahl],
    'expiry': expiry,
    'expiry_1': expiry_1,
    'heute': heute,
    #"nextContract": expiry_1,
    'tage_bis_verfall': tage_bis_verfall,
    'stock_price': stock_price,
    'Minkurs': Minkurs,
    'Maxkurs': Maxkurs,
    'Schritte': Schritte,
    'DetailMin': DetailMin,
    'DetailMax': DetailMax,
    'delta' : delta,
}).T.reset_index().rename(columns={0:"Value", 'index': "Info"})
info_df
for excel_file in list_excel_files:
    with pd.ExcelWriter(excel_file,datetime_format="DD/MM/YYYY") as writer:
        info_df.to_excel(writer,sheet_name='infos',index=False)
        Ueberhaenge_df.to_excel(writer,sheet_name='Ueberhaenge',index=False)
        Summery_df.to_excel(writer,sheet_name='Summery',index=False)
        SummeryDetail_df.to_excel(writer,sheet_name='SummeryDetail',index=False)
        overview_df.to_excel(writer,sheet_name='Overview',index=False)
        HedgeBedarf_df.to_excel(writer,sheet_name='HedgeBedarf',index=False)
        HedgeBedarf1_df.to_excel(writer,sheet_name='HedgeBedarf+01',index=False)
        #vda_2m_df.to_excel(writer,sheet_name='VDAX-New 2M Hist',index=False)
        #euribor_3m_df.to_excel(writer,sheet_name='EURIBOR 3M',index=False)
        
print("Excel files have been exported.")
HedgeBedarf_df["Sum"] = HedgeBedarf_df.HedgeSum + HedgeBedarf1_df.HedgeSum
# Fifth stage
# Values to create to arrow
idx_closest = (HedgeBedarf_df.Basis - stock_price).abs().idxmin()
closest_Basis =  HedgeBedarf_df.loc[idx_closest,"Basis"]
closest_Sum = HedgeBedarf_df.loc[idx_closest,"Sum"]

x_axis_length = HedgeBedarf_df.Sum.max() - HedgeBedarf_df.Sum.min()
y_axis_length = HedgeBedarf_df.Basis.max() - HedgeBedarf_df.Basis.min()



min_Kontrakte = 5000
max_Differenz = 1000
prozentual = 0.2

# (CF > min_Kontrakte) AND (PF > min_Kontrakte) AND (ABS(CF - PF)  < MIN(PF,CF)*)
mask_highlights = (
    (Summery_df.openInterest_CF > min_Kontrakte) & 
    (Summery_df.openInterest_PF > min_Kontrakte) & 
    (
        (Summery_df.openInterest_CF - Summery_df.openInterest_PF).abs() < 
        (Summery_df[['openInterest_PF',"openInterest_CF"]].min(axis=1)*prozentual)))

################################## Hilights for Basis column ####################################

Summery_df["basis_color"] = "lavender"
Summery_df.loc[
    mask_highlights, "basis_color"] = "yellow"
col_basis_color = Summery_df.basis_color.to_numpy()
#################################################################################################

######################### Gradient of red and blue for Anderung column ##########################
aux = Summery_df.Änderung.reset_index()
aux['r'] = aux['g']  = aux['b'] = 255
aux.loc[aux.Änderung<0,'g'] = aux.loc[aux.Änderung<0,'b'] = 255 - 150*aux.Änderung/(aux.Änderung.min()) 
aux.loc[aux.Änderung>0,'g'] = aux.loc[aux.Änderung>0,'r'] = 255 - 150*aux.Änderung/(aux.Änderung.max())
aux = aux.astype(float)
rb_shades = np.array([f"rgb({aux.loc[i,'r']},{aux.loc[i,'g']},{aux.loc[i,'b']})" for i in range(aux.shape[0]) ])
col_anderung_color = rb_shades
#################################################################################################

############################# Colors for heute and last_day columns #############################
col_heute_color = posneg_binary_color(Summery_df.heute,"rgb(0, 204, 204)","rgb(77, 166, 255)")
col_last_day_color = posneg_binary_color(Summery_df.last_day,"rgb(0, 204, 204)","rgb(77, 166, 255)")
#################################################################################################

###################### Gradient of green and blue for Put anc Call columns ######################
col_put_color = colors = np.array(n_colors('rgb(214, 245, 214)', 'rgb(40, 164, 40)',
    20, colortype='rgb'))[scale_col_range(Summery_df.openInterest_PF,19)]
col_call_color = colors = np.array(n_colors('rgb(204, 224, 255)', 'rgb(0, 90, 179)',
    20, colortype='rgb'))[scale_col_range(Summery_df.openInterest_CF,19)]
#################################################################################################


Euribor 3M in 9/1/2023 is 3.770 %
vstoxx = 15.94 PTS
Excel files have been exported.


In [ ]:
tage_bis_verfall <= 5 

False